## Create Argonaut MedicationStatement Resource

*[from the Argonaut Guide](http://www.fhir.org/guides/argonaut/r2/StructureDefinition-argo-medicationstatement.html)* 

### Each MedicationStatement must have:

- a patient
- a date
- a status
- a medication

### Profile specific implementation guidance:

The MedicationStatement and MedicationRequest resources can represent a medication, using either a code or refer to a Medication resource.
When referencing a Medication resource, the resource may be contained or an external resource.
The server application can support one or both methods, but if the an external reference to Medication is used, the server SHALL support the include parameter for searching this element.
The client application must support both methods.
Additional guidance is provided below in the Search section and in the conformance resource for this profile


This Example uses the [Python SMART on FHIR client](https://github.com/smart-on-fhir/client-py) library to create a [MedicationStatement](http://hl7.org/fhir/DSTU2/medicationstatement.html) Resource.

###  Import modules

In [4]:
import fhirclient.models.medicationstatement as MS
import fhirclient.models.identifier as I
import fhirclient.models.coding as C
import fhirclient.models.codeableconcept as CC
import fhirclient.models.fhirdate as D
import fhirclient.models.extension as X
import fhirclient.models.meta as M
import fhirclient.models.valueset as VS
import fhirclient.models.reference as R
import fhirclient.models.period as PD
import fhirclient.models.parameters as PM

from json import dumps
from pprint import pprint
from requests import get, post, put
import os
from IPython.display import display, Markdown, HTML

### Global variables


In [5]:
out_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
#out_path = "C:/Users/Eric/Documents/Jan_2019_FHIR_Experience"
fhir_term_server = 'http://test.fhir.org/r4'
fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu2'

headers = {
'Accept':'application/fhir+json',
'Content-Type':'application/fhir+json'
}


type = 'medicationstatement'
profile = 'http://fhir.org/guides/argonaut/StructureDefinition/argo-{type}'


### Function Definitions

#### look up code and system to get display term server using  the  `$lookup` operation
example: 
```
GET [base]/CodeSystem/$lookup?system=http://loinc.org&code=1963-8
```

In [31]:
def get_display(sys, code): 
    params = {
    'system': sys,
    'code': code,
    }
    #   r = requests.get('https://httpbin.org/post', params = {'key':'value'}, header = {'key':'value'})
    r = get(f'https://rxnav.nlm.nih.gov/REST/rxcui/{code}')
    # print(r.status_code)
    if r.status_code < 300:
        return (r.text.split('<name>')[1]).split('</name>')[0]
    else:
        print (dumps(r.json(),indent=3))

### Instantiate Classes

In [15]:
medstatement = MS.MedicationStatement()
reference=R.Reference()
meta = M.Meta()
identifier = I.Identifier()
codeableconcept = CC.CodeableConcept()
coding = C.Coding()
period = PD.Period()

display(Markdown('**Initial MedicationStatement resource as a Python dictionary object:**'))
display(Markdown(f'`{medstatement.as_json()}`'))


**Initial MedicationStatement resource as a Python dictionary object:**

`{'resourceType': 'MedicationStatement'}`

### Supply Data for Resource 

In this example, a simple Python dictionary of key:value pairs is being used, but you could extend this to grab the data from a spreadsheet, csv file, database, User entry, etc.

In [21]:
data = dict(
patient = 'test-1',
f_name = 'Eric',
l_name = "Haas",
mi = 'M',
date_asserted = '2019-01-08',
effective_hi= '2018-12-28',
effective_lo= '2018-12-14',
effective = '',
status = 'completed',
medication_code = '206765', #RX-Norm
# medication_reference = '#my_meds'
)


### Build Resource Using This Data
A step by step example to show how to create the resource using the FHIRClient Model.

#### Add simple id and status elements

In [23]:
medstatement.id = 'test-1'
medstatement.status = data['status']
pprint(medstatement.as_json())

{'id': 'test-1', 'resourceType': 'MedicationStatement', 'status': 'completed'}


#### Add complex meta element for  (repeating) profile element

In [24]:
meta.profile = []
meta.profile.append(f'http://fhir.org/guides/argonaut/StructureDefinition/argo-{type}')
medstatement.meta = meta
pprint(medstatement.as_json())


{'id': 'test-1',
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement']},
 'resourceType': 'MedicationStatement',
 'status': 'completed'}


#### Add patient reference element

In [25]:
reference.reference = f'Patient/{data["patient"]}'
reference.text = f'data["f_name"] data["mi"] data["l_name"]'
medstatement.patient = reference
pprint(medstatement.as_json())

{'id': 'test-1',
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement']},
 'patient': {'reference': 'Patient/test-1'},
 'resourceType': 'MedicationStatement',
 'status': 'completed'}


#### add dates

Note dates need to be converted to ISO formatted strings in Python

In [26]:
date = D.FHIRDate(data['date_asserted'])
medstatement.dateAsserted = date
period.start  = D.FHIRDate(data['effective_lo']) if data['effective_lo'] else None
period.end  = D.FHIRDate(data['effective_hi']) if data['effective_hi'] else None
medstatement.effectivePeriod = period
medstatement.effectiveDateTime =  D.FHIRDate(data['effective']) if data['effective'] else None
pprint(medstatement.as_json())

{'dateAsserted': '2019-01-08',
 'effectivePeriod': {'end': '2018-12-28', 'start': '2018-12-14'},
 'id': 'test-1',
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement']},
 'patient': {'reference': 'Patient/test-1'},
 'resourceType': 'MedicationStatement',
 'status': 'completed'}


#### Finaly, add the medication as a codeableconcept

In [33]:
sys = 'http://www.nlm.nih.gov/research/umls/rxnorm'
dis = get_display(sys,data['medication_code'])
codeableconcept.coding = []
coding.code = data['medication_code']
coding.system = sys
coding.display = dis
codeableconcept.coding.append(coding)
codeableconcept.text = dis
medstatement.medicationCodeableConcept = codeableconcept
pprint(medstatement.as_json())

{'dateAsserted': '2019-01-08',
 'effectivePeriod': {'end': '2018-12-28', 'start': '2018-12-14'},
 'id': 'test-1',
 'medicationCodeableConcept': {'coding': [{'code': '206765',
                                           'display': 'Lisinopril 10 MG Oral '
                                                      'Tablet [Prinivil]',
                                           'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}],
                               'text': 'Lisinopril 10 MG Oral Tablet '
                                       '[Prinivil]'},
 'meta': {'profile': ['http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement']},
 'patient': {'reference': 'Patient/test-1'},
 'resourceType': 'MedicationStatement',
 'status': 'completed'}


### Pretty print as json


In [34]:
print('\n',dumps(medstatement.as_json(),indent =3 ))



 {
   "id": "test-1",
   "meta": {
      "profile": [
         "http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement"
      ]
   },
   "dateAsserted": "2019-01-08",
   "effectivePeriod": {
      "end": "2018-12-28",
      "start": "2018-12-14"
   },
   "medicationCodeableConcept": {
      "coding": [
         {
            "code": "206765",
            "display": "Lisinopril 10 MG Oral Tablet [Prinivil]",
            "system": "http://www.nlm.nih.gov/research/umls/rxnorm"
         }
      ],
      "text": "Lisinopril 10 MG Oral Tablet [Prinivil]"
   },
   "patient": {
      "reference": "Patient/test-1"
   },
   "status": "completed",
   "resourceType": "MedicationStatement"
}


### Validate Resource

Note unable to locate an open test server to do this. :(

In [35]:
params = {
'profile': f'http://fhir.org/guides/argonaut/StructureDefinition/argo-{type}'
}

r = post(f'{fhir_test_server}/MedicationStatement/$validate', params = params, headers = headers, data = dumps(medstatement.as_json()))
# print(r.status_code)
# view  output in browser
display(Markdown(f'<h1>Validation output</h1>{r.json()["text"]["div"]}'))


<h1>Validation output</h1><div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">error</td><td>[]</td><td><pre>Failed to call access method: org.springframework.transaction.UnexpectedRollbackException: Transaction silently rolled back because it has been marked as rollback-only</pre></td>
					
				
			</tr>
		</table>
	</div>

### PUT Resource to FHIR test server

In [36]:
 
r = put(f'{fhir_test_server}/MedicationStatement/{medstatement.id}', headers = headers, data = dumps(medstatement.as_json()))


display(Markdown(f'<h1>Put Response</h1><p><strong>Transaction Status</strong> = {r.status_code}</p>'))

display(Markdown(f'<p>raw output :</p><code>{dumps(r.json(),indent=3)}</code>'))


<h1>Put Response</h1><p><strong>Transaction Status</strong> = 200</p>

<p>raw output :</p><code>{
   "resourceType": "MedicationStatement",
   "id": "test-1",
   "meta": {
      "versionId": "2",
      "lastUpdated": "2019-01-10T19:20:16.873+00:00",
      "profile": [
         "http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement"
      ]
   },
   "patient": {
      "reference": "Patient/test-1"
   },
   "dateAsserted": "2019-01-08",
   "status": "completed",
   "effectivePeriod": {
      "start": "2018-12-14",
      "end": "2018-12-28"
   },
   "medicationCodeableConcept": {
      "coding": [
         {
            "system": "http://www.nlm.nih.gov/research/umls/rxnorm",
            "code": "206765",
            "display": "Lisinopril 10 MG Oral Tablet [Prinivil]"
         }
      ],
      "text": "Lisinopril 10 MG Oral Tablet [Prinivil]"
   }
}</code>

### GET resource in XML format

In [39]:
headers_xml = {
'Accept':'application/fhir+xml',
'Content-Type':'application/fhir+xml'
}

#   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
r = get(f'{fhir_test_server}/MedicationStatement/{medstatement.id}', headers = headers_xml)
#print(r.status_code)
display(Markdown(f'<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = {r.status_code}</p>'))
print(r.text)



<h1>Get Resource as XML</h1><p><strong>Transaction Status</strong> = 200</p>

<MedicationStatement xmlns="http://hl7.org/fhir">
   <id value="test-1"/>
   <meta>
      <versionId value="2"/>
      <lastUpdated value="2019-01-10T19:20:16.873+00:00"/>
      <profile value="http://fhir.org/guides/argonaut/StructureDefinition/argo-medicationstatement"/>
   </meta>
   <patient>
      <reference value="Patient/test-1"/>
   </patient>
   <dateAsserted value="2019-01-08"/>
   <status value="completed"/>
   <effectivePeriod>
      <start value="2018-12-14"/>
      <end value="2018-12-28"/>
   </effectivePeriod>
   <medicationCodeableConcept>
      <coding>
         <system value="http://www.nlm.nih.gov/research/umls/rxnorm"/>
         <code value="206765"/>
         <display value="Lisinopril 10 MG Oral Tablet [Prinivil]"/>
      </coding>
      <text value="Lisinopril 10 MG Oral Tablet [Prinivil]"/>
   </medicationCodeableConcept>
</MedicationStatement>


## FIN

